# One-to-Many and Many-to-Many Joins - Lab

## Introduction

In this lab, you'll practice your knowledge on one-to-many and many-to-many relationships!

## Objectives

You will be able to:
- Query data using one-to-many and many-to-many joins
- Predict the resulting size of one-to-many and many-to-many joins

## One-to-Many and Many-to-Many Joins
<img src='images/Database-Schema.png' width="600">

## Connect to the Database

In [1]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('data.sqlite', detect_types=sqlite3.PARSE_COLNAMES)
cur = conn.cursor()

## Employees and their Office (a One-to-One join)

Return a list of all of the employees with their first name, last name and the city and state of the office that they work out of (if they have one). Include all employees and order them by their first name, then their last name.

In [6]:
cur.execute("""SELECT firstName, lastName, city, state
                      FROM employees e
                      JOIN offices o
                      USING (officeCode)
                      ORDER BY firstName""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,firstName,lastName,city,state
0,Andy,Fixter,Sydney,
1,Anthony,Bow,San Francisco,CA
2,Barry,Jones,London,
3,Diane,Murphy,San Francisco,CA
4,Foon Yue,Tseng,NYC,NY
5,George,Vanauf,NYC,NY
6,Gerard,Bondur,Paris,
7,Gerard,Hernandez,Paris,
8,Jeff,Firrelli,San Francisco,CA
9,Julie,Firrelli,Boston,MA


## Customers and their Orders (a One-to-Many join)

Return a list of all the customers first and last names along with a record for each of their order numbers, order dates and statuses.

In [8]:
cur.execute("""SELECT * from customers""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",,Nantes,,44000,France,1370,21000.00
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,,Las Vegas,NV,83030,USA,1166,71800.00
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611,117300.00
3,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",,Nantes,,44000,France,1370,118200.00
4,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,,Stavern,,4110,Norway,1504,81700.00


In [9]:
cur.execute("""SELECT * from orders""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head()

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,,363
1,10101,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128
2,10102,2003-01-10,2003-01-18,2003-01-14,Shipped,,181
3,10103,2003-01-29,2003-02-07,2003-02-02,Shipped,,121
4,10104,2003-01-31,2003-02-09,2003-02-01,Shipped,,141


In [11]:
cur.execute("""SELECT contactFirstName, contactLastName, orderNumber, orderDate, status
                      FROM customers 
                      JOIN orders 
                      USING (customerNumber)
                      ORDER BY contactFirstName""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head()

,contactFirstName,contactLastName,orderNumber,orderDate,status
0,Adrian,Huxley,10139,2003-07-16,Shipped
1,Adrian,Huxley,10270,2004-07-19,Shipped
2,Adrian,Huxley,10361,2004-12-17,Shipped
3,Adrian,Huxley,10420,2005-05-29,In Process
4,Akiko,Shimamura,10258,2004-06-15,Shipped


## Customers and their Payments (another One-to-Many join)

Return a list of customers first and last names along with details about their payments including the amount and date of payments. Sort these results in descending order by the payment amount.

In [12]:
cur.execute("""SELECT * from payments""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head()

,customerNumber,checkNumber,paymentDate,amount
0,103,HQ336336,2004-10-19,6066.78
1,103,JM555205,2003-06-05,14571.44
2,103,OM314933,2004-12-18,1676.14
3,112,BO864823,2004-12-17,14191.12
4,112,HQ55022,2003-06-06,32641.98


In [13]:
cur.execute("""SELECT contactFirstName, contactLastName, amount, paymentDate
                      FROM customers 
                      JOIN payments
                      USING (customerNumber)
                      ORDER BY amount DESC""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head()

,contactFirstName,contactLastName,amount,paymentDate
0,Violeta,Benitez,9977.85,2003-11-08
1,Ben,Calaghan,9821.32,2003-10-17
2,Leslie,Taylor,9658.74,2004-12-06
3,Sean,Clenahan,9415.13,2004-07-28
4,Roland,Mendel,8807.12,2005-05-03


## Orders, Order details and Product Details (a Many-to-Many Join)

Return a list of customer first and last names, product names, quantities, and date ordered for each of the customers and each of their orders. Sort these in descending order by the order date.

Note: This will require joining 4 tables! This can be tricky! Give it a shot, and if you're still stuck, turn to the next section where you'll see how to write subqueries which can make complex queries such as this much simpler!

In [19]:
cur.execute("""SELECT contactFirstName, contactLastName, productName, quantityOrdered,
                      orderDate
                      FROM customers 
                      JOIN orders
                      USING (customerNumber)
                      JOIN orderdetails
                      USING (orderNumber)
                      JOIN products
                      USING (productCode)
                      ORDER BY orderDate DESC;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head()

,contactFirstName,contactLastName,productName,quantityOrdered,orderDate
0,Janine,Labrune,1962 LanciaA Delta 16V,38,2005-05-31
1,Janine,Labrune,1957 Chevy Pickup,33,2005-05-31
2,Janine,Labrune,1998 Chrysler Plymouth Prowler,28,2005-05-31
3,Janine,Labrune,1964 Mercedes Tour Bus,38,2005-05-31
4,Janine,Labrune,1926 Ford Fire Engine,19,2005-05-31


## Summary

In this lab, you practiced your knowledge on one-to-many and many-to-many relationships!